In [6]:
load("util.sage")
B1 = load('blocos/bloco_1_linear_cubo'); ##Isso na formula  6.28 secao 6.4.1

B1 = B1.apply_map(lambda x: x.simplify_real().combine())
#show(B1)

<ipython-input-6-d978371e27a2>:2: DeprecationWarning: PY_SSIZE_T_CLEAN will be required for '#' formats
  B1 = load('blocos/bloco_1_linear_cubo'); ##Isso na formula  6.28 secao 6.4.1


In [7]:
p1_t = B1.det();# show(p1_t)

In [11]:
α0 = p1_t.coefficients()[0][0]
α1 = p1_t.coefficients()[1][0]

In [12]:
t = α0.variables()[0]


In [13]:
#α0_new  = α0.simplify_real().combine()

In [14]:
#α0_numerator = α0_new.numerator()

In [15]:
B11 = B1[0,0].subs(c = -α0/α1)

In [12]:
B11.numerator()(t=0)

0

In [13]:

B11_new = B11.numerator().simplify_real(); B11_new(t=0)

0

In [16]:
var('u,v')
assume(u>0)
assume(v>0)
B11_uv_t = B11_new.subs({(3*t^2 - 2*t + 3):v^2}).subs({(3*t^2 + 2*t + 3):u^2}).subs(t=(u^2-v^2)/4).simplify_real().combine()

In [17]:
B11_uv_t = B11_uv_t.expand()

In [18]:
#Resolve 3*t^2 + 2*t + 3=u^2 em relação a u
sol_u = solve(u**2 == 3*t^2 + 2*t + 3, u)[0].rhs() 

#Resolve 3*t^2 - 2*t + 3=v^2 em relação a v
sol_v = solve(v**2 == 3*t^2 - 2*t + 3, v)[0].rhs()

# obtem os valores de u para t=0 e t=1
u0 = sol_u(t=0)
u1 = sol_u(t=1)

# obtem os valores de v para t=1/3 e t=1
v0 = sol_v(t=1/3)
v1 = sol_v(t=1)

u0, u1, v0, v1

(sqrt(3), 2*sqrt(2), 2*sqrt(2/3), 2)

In [ ]:
#%time new_poly_u_v = subs_mobius_2d_uv(B11_uv_t, u0, u1, v0, v1)

In [ ]:
#has_signal_variations_u_v(new_poly_u_v)

In [15]:
graus_u_v_t  =  [poly_u_v_ico_num.degree(u), poly_u_v_ico_num.degree(v), poly_u_v_ico_num.degree(t)]
graus_u_v_t

[3, 3, 24]

In [16]:
from functools import reduce

def dicionario_geral(poly):
    """INPUT - poly- Um polinomio como expressão simbolica na classe <class 'sage.symbolic.expression.Expression'>
    OUTPUT - Um dicionario com CHAVE do tipo tupla multigrau (d1,d2,...,dl) correpondendo ao monomio 
    variavel1^(d1)*variavel2^(d2)*...*variavell^(dl) e VALOR sendo o coeficiente deste monomio.
    OBSERVACAO- A ordem das variaveis imspota pelo SAGE manda na ordem das chaves do dicionario. 
    Exemplo-
    sage: z, y, x, u = var('z y x u')
    sage: p = (2100*u^2*x+2020*u^2*y^2)*z^0+(3031*u^3*y^3+1101*u*x)*z+(3022*u^3*y^2+202*x^2)*z^2
    sage: variaveis = p.free_variables(); variaveis
        (u,x,y,z)
    sage: dicionario_geral(p)
        {(0, 2, 0, 2): 202,
         (1, 1, 0, 1): 1101,
         (2, 0, 2, 0): 2020,
         (2, 1, 0, 0): 2100,
         (3, 0, 2, 2): 3022,
         (3, 0, 3, 1): 3031}
        
        
    sage: u,v = var('u v')
    sage: p3 = 23*u^2*v^3+21*u^2*v+11*u*v+10*u+4;
    sage: dicionario_geral(p3)
      {(0, 0): 4, (1, 0): 10, (1, 1): 11, (2, 1): 21, (2, 3): 23}
    """
    def nova_lista(lista, var):
        new_lista = [];
        for item in lista:
            sublista = item[0].coefficients(var)
            for subitem in sublista:
                subitem = [subitem[0],subitem[1:]+item[-1]]
                new_lista.append(subitem)
        return  new_lista

    #variaveis = (poly).free_variables(); Modifiquei aqui
    variaveis = (poly).variables()
#####################################################################
    variavel_1 = variaveis[0];
    l_variavel_1 = (poly).coefficients(variavel_1);
    l = [[item[0],item[1:]] for item in l_variavel_1]
    for var in variaveis[1:]:
        l = nova_lista(l, var)
    dicionario = {tuple(list(reversed(j[1]))) : j[0] for j in l}
    return dicionario

def coeficientes_de_moebius_simbolico_varias_variaveis(poly, graus, extremos, novo_coefsxy ):
    """INPUTS -
    poly- Um polinomio como expressão simbolica na classe  <class 'sage.symbolic.expression.Expression'>;
    graus - Uma lista com naturais representando o multigrau do monomio que voce quer o coeficiente.
            Importante - A sua lista de graus deve estar na ordem que o Sage ordena as variaveis.
    extremos - Uma lista com pares (a_i,b_i) que dão o intervalo da substituição de Moebius para
    a variavel x_i, isto é,    x_i---> x_i = (a_i*u_i+b_i)/(u_i+1) para a variavel u_i.
    OUTPUT - O coeficiente do monomio com multigrau dado pela lista graus depois da substituicao de Moebius
    Exemplo - 1
    sage: poly_teste = 23*u^2*v^3 + 21*u^2*v + 11*u*v + 10*u + 4;
    sage: graus_teste = [1,2]; ## para acahar o coeficiente de (x_1)^1*(x_2)^2
    sage: extremos_teste = [(3, 5), (7, 11)];
    sage: novo_coefsxy_teste = dicionario_geral(poly_teste); # virar variável global;
    sage: c = coeficientes_de_moebius_simbolico_varias_variaveis(poly_teste, graus_teste, extremos_teste, novo_coefsxy_teste)
        1133944
    Exemplo - 2
    sage: poly = (2100*u^2*x+2020*u^2*y^2)*z^0+(3031*u^3*y^3+1101*u*x)*z+(3022*u^3*y^2+202*x^2)*z^2
    sage: variaveis = poly.free_variables(); variaveis
    sage: extremos_teste_2 = [(3,5),(7,11),(13,17),(19,23)];
    sage: ru, rx, ry, rz = var('ru, rx, ry, rz')
    sage: variaveis_r = [ru, rx, ry, rz]
    sage: novo_coefsxy_teste_2 = dicionario_geral(poly); 
    sage: c = coeficientes_de_moebius_simbolico_varias_variaveis(poly, [0, 0, 1, 0], extremos_teste_2, novo_coefsxy_teste_2) ## Procurando o coeficiente de ru^0 * rx^0 * ry^1 * rz^0
    sage: show(c)
     244374065874
       
       OBSERVACAO: Para mais detalhes veja o arquivo /Artigo_de_Aplicacoes/Reuniao-casa-marcelo-26-07/Icosaedro-bloco-2-reunião-casa-marcelo/Funcao_Para_Coeficientes_Com_Varias_Variaveis_Versao_Final.ipynb
        """
    #variaveis = (poly).free_variables(); Modifiquei aqui
    variaveis = (poly).variables()
    graus_total = [poly.degree(x) for x in variaveis];
    chaves = novo_coefsxy.keys()
    soma_total = 0
    for chave in chaves:
        somas = []
        for k in range(len(chave)):
            somak = 0
            ck = chave[k]
            gt = graus_total[k]
            gks = graus[k]
            a_k = extremos[k][0]
            b_k = extremos[k][1]
            for pp in range(ck+1):
                somak = somak+((binomial(ck,pp))*(binomial(gt-ck, gks-pp))*((a_k)^(pp))*((b_k)^(ck-pp)))
            somas.append(somak)
        somas_e_valor = somas + [novo_coefsxy[chave]]
        soma_total = soma_total + reduce(lambda a, b: a*b, somas_e_valor)
    return soma_total




In [17]:
dicionario = dicionario_geral(poly_u_v_ico_num)

In [18]:
dicionario_em_AA = {tuple([ZZ(k) for k in key]) : AA(value) for key, value in  dicionario.items()}


In [19]:
n_uplas = [[i,j,k] for i in range(graus_u_v_t[0]+1) for j in range(graus_u_v_t[1]+1) for k in range(graus_u_v_t[2]+1)]

In [20]:
extremos_u_v_ico_bloco_2_em_AA = [(AA(u0), AA(u1)), (AA(v0), AA(v1)), (0, 1)]


In [21]:
%time coeficientes_em_AA = [coeficientes_de_moebius_simbolico_varias_variaveis(poly_u_v_ico_num, grau, extremos_u_v_ico_bloco_2_em_AA, dicionario_em_AA ) for grau in n_uplas]

CPU times: user 54.1 s, sys: 0 ns, total: 54.1 s
Wall time: 54.1 s


In [22]:
numericos = [sign(c) for c in coeficientes_em_AA]

In [23]:
sinal = set([sign(c) for c in coeficientes_em_AA])

In [24]:
sinal

{-1, 0}